## Steps to create a React agent using MCP Server

### Create a mcp_client to connnect to the MCP Server

### Create a LLM

### Create the LangGraph StateGraph
- Need a State for sharing between nodes
- Create required nodes
- First node to be model node
- Second is tool node
- Conditional node to handle tool calling

In [ ]:
# Step 1: Set up MCP client
import os
import asyncio
from langchain_mcp_adapters.client import MultiServerMCPClient

BEARER_TOKEN = os.getenv("MCP_BEARER_TOKEN")
connections = {
    "github_mcp_server": {
        "url": "https://api.githubcopilot.com/mcp/",
        "transport": "streamable_http",
        "headers": {"Authorization": f"Bearer {BEARER_TOKEN}"},
    }
}
client = MultiServerMCPClient(connections)
tools = await client.get_tools()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Login to HF using token
from huggingface_hub import login
HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")
login(HUGGINGFACE_TOKEN)

# Load the Mistral model with Hugging Face Transformers
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Wrap the model in a LangChain LLM class
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline

hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

In [ ]:
# Create a LangGraph State
from langchain_core.messages import SystemMessage

# Graph Imports
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.constants import START, END


# Create Nodes for the graph
async def llm_with_tools(state: MessagesState) -> dict:
    messages = state["messages"]
    try:
        response = await llm.bind_tools(tools).ainvoke(messages)
    except Exception as e:
        print("Error during LLM invocation:", e)
        raise
    return {"messages": [response]}


graph_builder = StateGraph(state_schema=MessagesState)
graph_builder.add_node("llm_with_tools", llm_with_tools)
graph_builder.add_node("tools", ToolNode(tools))

# Define edges
graph_builder.add_edge(START, "llm_with_tools")
graph_builder.add_edge("tools", "llm_with_tools")
graph_builder.add_conditional_edges(
    "llm_with_tools", tools_condition, {"tools": "tools", "__end__": END}
)

# compile graph
agent = graph_builder.compile(name="GitHub Agent", debug=True)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

system_prompt = SystemMessage(
    content="""
        You are a GitHub Agent designed to interact with public GitHub repositories.
        Your task is to assist users with their queries related to GitHub repositories by utilizing the available tools effectively.
        """
)
user_prompt = HumanMessage(
    content="""
        Read the README.md from a repository and provide me a summary. Details owner: baliyanvinay, repo: Data-Structures-and-Algorithms
        """
)

# Run the agent
resp = await agent.ainvoke({"messages": [system_prompt, user_prompt]})
print("________________________LLM Response________________________")
print(resp["messages"][-1].content)